<a href="https://colab.research.google.com/github/globalenglish01/Billion/blob/main/AddChineseVioceForVedio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------------- 1️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts moviepy pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 2️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ---------------- 3️⃣ 导入库 ----------------
import yt_dlp
import os
import re
import concurrent.futures

# ---------------- 4️⃣ 设置 Cookie 文件 ----------------
cookie_path = "/content/drive/MyDrive/cookies.txt"  # 如果需要登录才能下载，请填入 Cookie 文件路径

# ---------------- 5️⃣ 短链接列表 ----------------
"""
video_links = [
    "https://b23.tv/OH0xxfs",
    "https://b23.tv/oivfFpK",
    "https://b23.tv/MI4fyMd",
    "https://b23.tv/oYi0svC",
    "https://b23.tv/BeUSQMD",
    "https://b23.tv/bkNOJGN",
    "https://b23.tv/grLGykZ",
    "https://b23.tv/sT4n4N7",
    "https://b23.tv/9bwXp5R",
    "https://b23.tv/V2t4Xtj",
    "https://b23.tv/qPCPUZj",
    "https://b23.tv/lI2oJZL",
    "https://b23.tv/kXUksZf",
    "https://b23.tv/LHXoFZW",
    "https://b23.tv/dkoeI7v"
]
"""

#【Udemy - Apache Airflow The Hands-On Guide-哔哩哔哩】 https://b23.tv/9Hi6puI
#【Udemy - The Complete Hands-On Introduction to Apache Airflow 3-哔哩哔哩】 https://b23.tv/pCsHwg6
#【Udemy - Snowflake – The Complete Masterclass part1-哔哩哔哩】 https://b23.tv/cWYeaK3
#【Udemy - Snowflake – The Complete Masterclass part2-哔哩哔哩】 https://b23.tv/GgDSO7s
#【Udemy - Ultimate AWS Certified Solutions Architect Associate SAA-C03 part1-哔哩哔哩】 https://b23.tv/P6rrSMd
#【Udemy - Ultimate AWS Certified Solutions Architect Associate SAA-C03 part2-哔哩哔哩】 https://b23.tv/wk8RQdA
#【Udemy - Ultimate AWS Certified Solutions Architect Associate SAA-C03 part3-哔哩哔哩】 https://b23.tv/LuxNkrW
#【Udemy - Ultimate AWS Certified Solutions Architect Associate SAA-C03 part4-哔哩哔哩】 https://b23.tv/h0uWaCQ
video_links = [
    "https://b23.tv/9Hi6puI",
    "https://b23.tv/pCsHwg6",
    "https://b23.tv/cWYeaK3",
    "https://b23.tv/GgDSO7s",
    "https://b23.tv/P6rrSMd",
    "https://b23.tv/wk8RQdA",
    "https://b23.tv/LuxNkrW",
    "https://b23.tv/h0uWaCQ"
]



# ---------------- 6️⃣ 提取完整 URL ----------------
video_urls = []

for link in video_links:
    ydl_opts = {
        "quiet": True,
        "skip_download": True,
        "extract_flat": True,
        "forcejson": True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(link, download=False)
        if isinstance(info, dict) and "url" in info:
            full_url = info["url"]
            clean_url = full_url.split('/?')[0]
            print(f"✅ 提取 URL: {clean_url}")
            video_urls.append(clean_url)
        else:
            print(f"⚠️ 未提取到 URL: {link}")

# ---------------- 7️⃣ 下载函数（单个 URL） ----------------
save_dir = "/content/drive/MyDrive/bilibili"
os.makedirs(save_dir, exist_ok=True)

def download_video(url):
    try:
        # 先获取视频信息
        ydl_opts = {
            "skip_download": True,
            "cookies": cookie_path,
            "forcejson": True,
            "noplaylist": False
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)

            entries = info.get("entries", [info])
            for entry in entries:
                if not entry:
                    continue
                idx = entry.get("playlist_index", "")
                title = entry.get("title", "untitled")
                ext = entry.get("ext", "mp4")

                # 处理非法文件名
                safe_title = re.sub(r'[\\/*?:"<>|]', "_", title)

                video_file = f"{save_dir}/{idx}_{safe_title}.{ext}"
                subtitle_file = f"{save_dir}/{idx}_{safe_title}.ai-zh.srt"

                skip_video = os.path.exists(video_file)
                skip_subs = os.path.exists(subtitle_file)

                if skip_video and skip_subs:
                    print(f"⏩ 跳过: {video_file}")
                    continue

                print(f"▶️ 开始下载: {url} (P{idx})")
                os.system(
                    f'yt-dlp --verbose --no-overwrites --cookies "{cookie_path}" --yes-playlist '
                    f'--write-subs --sub-lang ai-zh --sub-format srt --convert-subs srt '
                    f'--compat-options no-keep-subs '
                    f'-o "{save_dir}/%(playlist_index)s_%(title)s.%(ext)s" "{url}"'
                )
                print(f"✅ 完成下载: {url} (P{idx})")

    except Exception as e:
        print(f"❌ 下载失败: {url}, 错误: {e}")

# ---------------- 8️⃣ 多线程并行下载 ----------------
max_workers = min(5, len(video_urls))  # 限制最大并发 5
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(download_video, url) for url in video_urls]
    concurrent.futures.wait(futures)

print("🎉 全部任务完成！")

In [ ]:
!ls /content/drive/MyDrive/bilibili

In [ ]:
# ---------------- 3️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 1️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import io
import asyncio
import pysrt
import edge_tts
from pydub import AudioSegment
import random
import subprocess

# ---------------- 2️⃣ 保存目录 ----------------
save_dir = "/content/drive/MyDrive/bilibili"
os.makedirs(save_dir, exist_ok=True)

# ---------------- 5️⃣ 可选声音 ----------------
voices = [
    "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "zh-CN-YunjianNeural",
    "zh-CN-YunyangNeural", "zh-CN-XiaochenNeural", "zh-CN-XiaohanNeural",
    "zh-CN-XiaomoNeural", "zh-CN-XiaoruiNeural", "zh-CN-XiaoqiuNeural",
    "zh-CN-XiaoshuangNeural", "zh-CN-XiaoyouNeural",
    "zh-CN-XiaozhenNeural", "zh-CN-YunfengNeural", "zh-CN-YunhaoNeural",
    "zh-CN-YunxiaNeural", "zh-CN-YunzeNeural", "zh-CN-XiaorouNeural",
    "zh-CN-XiaoxuanNeural", "zh-CN-XiaoyiNeural", "zh-CN-XiaomengNeural"
]

# ---------------- 6️⃣ 限流+重试生成单句 TTS ----------------
semaphore = asyncio.Semaphore(2)  # 同时最多 2 个请求

async def generate_tts_segment_with_retry(text, voice, retries=3):
    if not text.strip():
        return AudioSegment.silent(duration=500)
    for attempt in range(retries):
        try:
            async with semaphore:
                communicate = edge_tts.Communicate(text, voice=voice)
                audio_bytes = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_bytes += chunk["data"]
                return AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")
        except Exception as e:
            if "429" in str(e):
                wait = 2 ** attempt
                print(f"⚠️ 429 限流，等待 {wait}s 重试...")
                await asyncio.sleep(wait)
            else:
                raise e
    raise Exception("TTS 多次重试失败")

# ---------------- 7️⃣ 并发生成完整音频 ----------------
"""
async def generate_full_tts_concurrent(srt_path, voice):
    subs = pysrt.open(srt_path)
    clips = [None]*len(subs)
    async def worker(i, text):
        clips[i] = await generate_tts_segment_with_retry(text, voice)
    tasks = [worker(i, sub.text.strip()) for i, sub in enumerate(subs)]
    await asyncio.gather(*tasks)
    return sum(clips)
"""
# ---------------- 7️⃣ 并发生成完整音频 (去重字幕) ----------------
async def generate_full_tts_concurrent(srt_path, voice):
    subs = pysrt.open(srt_path)

    # 去掉连续重复的字幕
    unique_subs = []
    prev_text = None
    for sub in subs:
        text = sub.text.strip()
        if text and text != prev_text:  # 跳过连续重复
            unique_subs.append(text)
        prev_text = text

    clips = [None] * len(unique_subs)

    async def worker(i, text):
        clips[i] = await generate_tts_segment_with_retry(text, voice)

    tasks = [worker(i, text) for i, text in enumerate(unique_subs)]
    await asyncio.gather(*tasks)

    # 拼接所有音频片段
    return sum(clips)

# ---------------- 8️⃣ 调整音频长度 ----------------
def adjust_audio_length_ffmpeg(input_path, output_path, target_duration):
    probe = subprocess.run(
        ["ffprobe", "-i", input_path, "-show_entries", "format=duration",
         "-v", "quiet", "-of", "csv=p=0"],
        capture_output=True, text=True
    )
    orig_dur = float(probe.stdout.strip())
    speed = orig_dur / target_duration
    filters = []
    while speed > 2.0:
        filters.append("atempo=2.0")
        speed /= 2.0
    while speed < 0.5:
        filters.append("atempo=0.5")
        speed /= 0.5
    filters.append(f"atempo={speed}")
    subprocess.run(["ffmpeg","-y","-i",input_path,"-filter:a",",".join(filters),output_path])

# ---------------- 9️⃣ 合成视频 ----------------
def merge_video_audio_ffmpeg(video_file, audio_file, output_file):
    subprocess.run([
        "ffmpeg","-y","-i",video_file,"-i",audio_file,
        "-map","0:v:0","-map","1:a:0",
        "-c:v","copy","-c:a","aac",output_file
    ])

# ---------------- 🔟 批量处理 ----------------
async def process_videos():
    video_files = [f for f in os.listdir(save_dir) if f.endswith(".mp4")]
    for vf in sorted(video_files):
        base = os.path.splitext(vf)[0]
        srt_file = os.path.join(save_dir, base+".ai-zh.srt")
        if not os.path.exists(srt_file):
            print(f"❌ 跳过 {vf}，没有 SRT 文件。")
            continue

        video_file = os.path.join(save_dir,vf)
        raw_audio = os.path.join(save_dir,base+"_raw.mp3")
        adjusted_audio = os.path.join(save_dir,base+"_tts.mp3")
        output_file = os.path.join(save_dir,base+"_CN.mp4")

        voice = random.choice(voices)
        print(f"▶️ 正在处理 {vf}，声音: {voice}")

        final_audio = await generate_full_tts_concurrent(srt_file, voice)
        final_audio.export(raw_audio, format="mp3")

        probe = subprocess.run(
            ["ffprobe","-i",video_file,"-show_entries","format=duration",
             "-v","quiet","-of","csv=p=0"],
            capture_output=True, text=True
        )
        video_dur = float(probe.stdout.strip())
        adjust_audio_length_ffmpeg(raw_audio, adjusted_audio, video_dur)
        merge_video_audio_ffmpeg(video_file, adjusted_audio, output_file)
        print(f"✅ 完成 {vf}")

# ---------------- 1️⃣1️⃣ 执行 ----------------
await process_videos()

In [ ]:

# ---------------- 3️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 1️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import io
import asyncio
import pysrt
import edge_tts
from pydub import AudioSegment
import random
import subprocess
import shutil

# ---------------- 2️⃣ 保存目录 ----------------
save_dir = "/content/drive/MyDrive/bilibili"         # 原始目录
output_dir = os.path.join(save_dir, "output")        # 成品目录
os.makedirs(save_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# ---------------- 5️⃣ 可选声音 ----------------
voices = [
    "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "zh-CN-YunjianNeural",
    "zh-CN-YunyangNeural", "zh-CN-XiaochenNeural", "zh-CN-XiaohanNeural",
    "zh-CN-XiaomoNeural", "zh-CN-XiaoruiNeural", "zh-CN-XiaoqiuNeural",
    "zh-CN-XiaoshuangNeural", "zh-CN-XiaoyouNeural",
    "zh-CN-XiaozhenNeural", "zh-CN-YunfengNeural", "zh-CN-YunhaoNeural",
    "zh-CN-YunxiaNeural", "zh-CN-YunzeNeural", "zh-CN-XiaorouNeural",
    "zh-CN-XiaoxuanNeural", "zh-CN-XiaoyiNeural", "zh-CN-XiaomengNeural"
]

# ---------------- 6️⃣ 限流+重试生成单句 TTS ----------------
semaphore = asyncio.Semaphore(2)  # 同时最多 2 个请求

async def generate_tts_segment_with_retry(text, voice, output_path, retries=3):
    """生成单句 TTS，保存为 mp3 文件"""
    if os.path.exists(output_path):
        return  # 断点续传：已存在则跳过

    if not text.strip():
        AudioSegment.silent(duration=500).export(output_path, format="mp3")
        return

    for attempt in range(retries):
        try:
            async with semaphore:
                communicate = edge_tts.Communicate(text, voice=voice)
                audio_bytes = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_bytes += chunk["data"]
                AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3").export(output_path, format="mp3")
                return
        except Exception as e:
            if "429" in str(e):
                wait = 2 ** attempt
                print(f"⚠️ 429 限流，等待 {wait}s 重试...")
                await asyncio.sleep(wait)
            else:
                raise e
    raise Exception("TTS 多次重试失败")

# ---------------- 7️⃣ 并发生成完整音频 ----------------
async def generate_full_tts_concurrent(srt_path, voice, base_name):
    subs = pysrt.open(srt_path)

    # 去掉连续重复字幕
    unique_subs = []
    prev_text = None
    for sub in subs:
        text = sub.text.strip()
        if text and text != prev_text:
            unique_subs.append(text)
        prev_text = text

    # 分片目录
    parts_dir = os.path.join(output_dir, "parts", base_name)
    os.makedirs(parts_dir, exist_ok=True)

    part_files = []
    tasks = []
    for i, text in enumerate(unique_subs):
        part_file = os.path.join(parts_dir, f"{base_name}_part_{i:04d}.mp3")
        part_files.append(part_file)
        tasks.append(generate_tts_segment_with_retry(text, voice, part_file))

    await asyncio.gather(*tasks)

    # concat 文件
    concat_list = os.path.join(parts_dir, f"{base_name}_concat.txt")
    with open(concat_list, "w", encoding="utf-8") as f:
        for pf in part_files:
            f.write(f"file '{pf}'\n")

    # 拼接
    final_audio_file = os.path.join(output_dir, f"{base_name}_raw.mp3")
    subprocess.run([
        "ffmpeg", "-y", "-f", "concat", "-safe", "0",
        "-i", concat_list, "-c", "copy", final_audio_file
    ])

    # 删除分片目录
    shutil.rmtree(parts_dir, ignore_errors=True)

    return final_audio_file

# ---------------- 8️⃣ 调整音频长度 ----------------
def adjust_audio_length_ffmpeg(input_path, output_path, target_duration):
    probe = subprocess.run(
        ["ffprobe", "-i", input_path, "-show_entries", "format=duration",
         "-v", "quiet", "-of", "csv=p=0"],
        capture_output=True, text=True
    )
    orig_dur = float(probe.stdout.strip())
    speed = orig_dur / target_duration
    filters = []
    while speed > 2.0:
        filters.append("atempo=2.0")
        speed /= 2.0
    while speed < 0.5:
        filters.append("atempo=0.5")
        speed /= 0.5
    filters.append(f"atempo={speed}")
    subprocess.run(["ffmpeg","-y","-i",input_path,"-filter:a",",".join(filters),output_path])

# ---------------- 9️⃣ 合成视频 ----------------
def merge_video_audio_ffmpeg(video_file, audio_file, output_file):
    subprocess.run([
        "ffmpeg","-y","-i",video_file,"-i",audio_file,
        "-map","0:v:0","-map","1:a:0",
        "-c:v","copy","-c:a","aac",output_file
    ])

# ---------------- 🔟 批量处理 ----------------
async def process_videos():
    video_files = [f for f in os.listdir(save_dir) if f.endswith(".mp4")]
    for vf in sorted(video_files):
        base = os.path.splitext(vf)[0]
        srt_file = os.path.join(save_dir, base + ".ai-zh.srt")

        output_video = os.path.join(output_dir, base + "_CN.mp4")
        output_audio = os.path.join(output_dir, base + "_tts.mp3")
        output_srt   = os.path.join(output_dir, base + "_CN.srt")

        if os.path.exists(output_video):
            print(f"⏭️ 跳过 {vf}，最终文件已存在。")
            continue

        if not os.path.exists(srt_file):
            print(f"❌ 跳过 {vf}，没有 SRT 文件。")
            continue

        video_file = os.path.join(save_dir, vf)

        voice = random.choice(voices)
        print(f"▶️ 正在处理 {vf}，声音: {voice}")

        raw_audio = await generate_full_tts_concurrent(srt_file, voice, base)

        # 视频时长
        probe = subprocess.run(
            ["ffprobe","-i",video_file,"-show_entries","format=duration",
             "-v","quiet","-of","csv=p=0"],
            capture_output=True, text=True
        )
        video_dur = float(probe.stdout.strip())

        adjust_audio_length_ffmpeg(raw_audio, output_audio, video_dur)
        merge_video_audio_ffmpeg(video_file, output_audio, output_video)

        # 复制字幕文件到 output 文件夹，并改名
        shutil.copy(srt_file, output_srt)

        print(f"✅ 完成 {vf}，输出文件位于 {output_dir}")

# ---------------- 1️⃣1️⃣ 执行 ----------------
await process_videos()

In [ ]:

# ---------------- 3️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 1️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import io
import asyncio
import pysrt
import edge_tts
from pydub import AudioSegment
import random
import subprocess
import shutil

# ---------------- 2️⃣ 保存目录 ----------------
save_dir = "/content/drive/MyDrive/bilibili"         # 原始目录
output_dir = os.path.join(save_dir, "output")        # 成品目录
os.makedirs(save_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# ---------------- 5️⃣ 可选声音 ----------------
voices = [
    "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "zh-CN-YunjianNeural",
    "zh-CN-YunyangNeural", "zh-CN-XiaochenNeural", "zh-CN-XiaohanNeural",
    "zh-CN-XiaomoNeural", "zh-CN-XiaoruiNeural", "zh-CN-XiaoqiuNeural",
    "zh-CN-XiaoshuangNeural", "zh-CN-XiaoyouNeural",
    "zh-CN-XiaozhenNeural", "zh-CN-YunfengNeural", "zh-CN-YunhaoNeural",
    "zh-CN-YunxiaNeural", "zh-CN-YunzeNeural", "zh-CN-XiaorouNeural",
    "zh-CN-XiaoxuanNeural", "zh-CN-XiaoyiNeural", "zh-CN-XiaomengNeural"
]

# ---------------- 6️⃣ 限流+重试生成单句 TTS ----------------
semaphore = asyncio.Semaphore(2)  # 同时最多 2 个请求

async def generate_tts_segment_with_retry(text, voice, output_path, retries=3):
    if os.path.exists(output_path):
        return  # 断点续传

    if not text.strip():
        AudioSegment.silent(duration=500).export(output_path, format="mp3")
        return

    for attempt in range(retries):
        try:
            async with semaphore:
                communicate = edge_tts.Communicate(text, voice=voice)
                audio_bytes = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_bytes += chunk["data"]
                AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3").export(output_path, format="mp3")
                return
        except Exception as e:
            if "429" in str(e):
                wait = 2 ** attempt
                print(f"⚠️ 429 限流，等待 {wait}s 重试...")
                await asyncio.sleep(wait)
            else:
                raise e
    raise Exception("TTS 多次重试失败")

# ---------------- 7️⃣ 处理单个字幕片段 ----------------
async def process_subtitle_segment(video_file, start_ms, end_ms, audio_file, output_file):
    # 提取对应视频段
    temp_video = output_file + "_video.mp4"
    subprocess.run([
        "ffmpeg", "-y",
        "-i", video_file,
        "-ss", str(start_ms/1000), "-to", str(end_ms/1000),
        "-c:v", "copy",
        "-an", temp_video
    ])
    # 对应音频长度
    audio_dur = AudioSegment.from_file(audio_file).duration_seconds
    video_dur = end_ms / 1000 - start_ms / 1000
    if audio_dur > video_dur:
        # 音频比视频长 → 在视频末尾补停帧
        freeze_file = output_file + "_freeze.mp4"
        subprocess.run([
            "ffmpeg", "-y",
            "-i", temp_video,
            "-vf", f"tpad=stop_mode=clone:stop_duration={audio_dur - video_dur}",
            "-an", freeze_file
        ])
        temp_video = freeze_file
    # 合并音频
    subprocess.run([
        "ffmpeg", "-y",
        "-i", temp_video,
        "-i", audio_file,
        "-c:v", "copy",
        "-c:a", "aac",
        output_file
    ])
    # 清理临时文件
    if os.path.exists(temp_video):
        os.remove(temp_video)
    if 'freeze_file' in locals() and os.path.exists(freeze_file):
        os.remove(freeze_file)

# ---------------- 8️⃣ 批量处理 ----------------
async def process_videos():
    video_files = [f for f in os.listdir(save_dir) if f.endswith(".mp4")]
    for vf in sorted(video_files):
        base = os.path.splitext(vf)[0]
        srt_file = os.path.join(save_dir, base + ".ai-zh.srt")
        output_video = os.path.join(output_dir, base + "_CN.mp4")
        output_srt   = os.path.join(output_dir, base + "_CN.srt")

        if os.path.exists(output_video):
            print(f"⏭️ 跳过 {vf}，最终文件已存在。")
            continue
        if not os.path.exists(srt_file):
            print(f"❌ 跳过 {vf}，没有 SRT 文件。")
            continue

        shutil.copy(srt_file, output_srt)
        video_file = os.path.join(save_dir, vf)
        voice = random.choice(voices)
        print(f"▶️ 正在处理 {vf}，声音: {voice}")

        subs = pysrt.open(srt_file)
        segments = []
        temp_audio_dir = os.path.join(output_dir, "temp_audio", base)
        temp_video_dir = os.path.join(output_dir, "temp_video", base)
        os.makedirs(temp_audio_dir, exist_ok=True)
        os.makedirs(temp_video_dir, exist_ok=True)

        for i, sub in enumerate(subs):
            text = sub.text.strip()
            start_ms = sub.start.ordinal
            end_ms = sub.end.ordinal

            # 连续字幕间隙
            if i > 0 and start_ms > subs[i-1].end.ordinal:
                gap_start = subs[i-1].end.ordinal
                gap_end = start_ms
                gap_audio = os.path.join(temp_audio_dir, f"gap_{i:04d}.mp3")
                AudioSegment.silent(duration=(gap_end-gap_start)).export(gap_audio, format="mp3")
                gap_video = os.path.join(temp_video_dir, f"gap_{i:04d}.mp4")
                subprocess.run([
                    "ffmpeg", "-y",
                    "-i", video_file,
                    "-ss", str(gap_start/1000), "-to", str(gap_end/1000),
                    "-an",
                    "-c:v", "copy",
                    gap_video
                ])
                gap_seg = os.path.join(temp_video_dir, f"gap_seg_{i:04d}.mp4")
                subprocess.run([
                    "ffmpeg", "-y",
                    "-i", gap_video,
                    "-i", gap_audio,
                    "-c:v", "copy",
                    "-c:a", "aac",
                    gap_seg
                ])
                segments.append(gap_seg)

            if not text:
                continue
            part_file = os.path.join(temp_audio_dir, f"{i:04d}.mp3")
            await generate_tts_segment_with_retry(text, voice, part_file)
            seg_output = os.path.join(temp_video_dir, f"seg_{i:04d}.mp4")
            await process_subtitle_segment(video_file, start_ms, end_ms, part_file, seg_output)
            segments.append(seg_output)

        # 拼接最终视频
        concat_file = os.path.join(temp_video_dir, "concat.txt")
        with open(concat_file, "w", encoding="utf-8") as f:
            for seg in segments:
                f.write(f"file '{seg}'\n")
        subprocess.run([
            "ffmpeg", "-y", "-f", "concat", "-safe", "0",
            "-i", concat_file, "-c", "copy", output_video
        ])
        shutil.rmtree(temp_audio_dir, ignore_errors=True)
        shutil.rmtree(temp_video_dir, ignore_errors=True)
        print(f"✅ 完成 {vf}，输出文件位于 {output_dir}")

# ---------------- 9️⃣ 执行 ----------------
await process_videos()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive
▶️ 正在处理 001_Udemy - Snowflake – The Complete Masterclass part1 p01 001 Welcome!.mp4，选择声音: zh-CN-XiaomoNeural


In [ ]:
# ---------------- 3️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 1️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import io
import asyncio
import pysrt
import edge_tts
from pydub import AudioSegment
import random
import subprocess
import shutil

# ---------------- 2️⃣ 保存目录 ----------------
save_dir = "/content/drive/MyDrive/bilibili"          # 原始目录
output_dir = os.path.join(save_dir, "output")         # 成品目录
os.makedirs(save_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# ---------------- 5️⃣ 可选声音 ----------------
voices = [
    "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "zh-CN-YunjianNeural",
    "zh-CN-YunyangNeural", "zh-CN-XiaochenNeural", "zh-CN-XiaohanNeural",
    "zh-CN-XiaomoNeural", "zh-CN-XiaoruiNeural", "zh-CN-XiaoqiuNeural",
    "zh-CN-XiaoshuangNeural", "zh-CN-XiaoyouNeural",
    "zh-CN-XiaozhenNeural", "zh-CN-YunfengNeural", "zh-CN-YunhaoNeural",
    "zh-CN-YunxiaNeural", "zh-CN-YunzeNeural", "zh-CN-XiaorouNeural",
    "zh-CN-XiaoxuanNeural", "zh-CN-XiaoyiNeural", "zh-CN-XiaomengNeural"
]

# ---------------- 6️⃣ 限流+重试生成单句 TTS ----------------
semaphore = asyncio.Semaphore(2)

async def generate_tts_segment(text, voice, output_path, duration_ms):
    """生成单句 TTS，并补充静音到 duration_ms"""
    if os.path.exists(output_path):
        return AudioSegment.from_file(output_path, format="mp3")

    if not text.strip():
        seg = AudioSegment.silent(duration=duration_ms)
        seg.export(output_path, format="mp3")
        return seg

    while True:
        try:
            async with semaphore:
                communicate = edge_tts.Communicate(text, voice=voice)
                audio_bytes = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_bytes += chunk["data"]
                seg = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")
                if len(seg) < duration_ms:
                    seg += AudioSegment.silent(duration=(duration_ms - len(seg)))
                seg.export(output_path, format="mp3")
                return seg
        except Exception as e:
            if "429" in str(e):
                await asyncio.sleep(2)
            else:
                raise e

# ---------------- 7️⃣ 生成每段视频+音频片段（可并发） ----------------
async def generate_segment(video_file, sub, voice, base_name, index, parts_dir):
    start_ms = int(sub.start.ordinal)
    end_ms = int(sub.end.ordinal)
    duration_ms = end_ms - start_ms

    part_audio_file = os.path.join(parts_dir, f"{base_name}_part_{index:04d}.mp3")
    tts_seg = await generate_tts_segment(sub.text.strip(), voice, part_audio_file, duration_ms)

    part_video_file = os.path.join(parts_dir, f"{base_name}_video_{index:04d}.mp4")
    subprocess.run([
        "ffmpeg", "-y", "-i", video_file,
        "-ss", str(start_ms / 1000.0),
        "-to", str(end_ms / 1000.0),
        "-c:v", "libx264", "-c:a", "aac",
        part_video_file
    ], check=True)

    video_dur_ms = end_ms - start_ms
    pause_ms = max(0, len(tts_seg) - video_dur_ms)

    if pause_ms > 0:
        tmp_file = part_video_file + "_tmp.mp4"
        cmd = [
            "ffmpeg", "-y", "-i", part_video_file,
            "-vf", f"tpad=stop_mode=clone:stop_duration={pause_ms/1000.0}",
            "-af", f"apad=pad_dur={pause_ms/1000.0}",
            "-c:v", "libx264", "-c:a", "aac", tmp_file
        ]
        subprocess.run(cmd, check=True)
        os.replace(tmp_file, part_video_file)

    return part_video_file, part_audio_file

# ---------------- 8️⃣ 并发生成完整视频 ----------------
async def generate_full_video(base_name, video_file, srt_file, voice):
    parts_dir = os.path.join(output_dir, "parts", base_name)
    os.makedirs(parts_dir, exist_ok=True)

    subs = pysrt.open(srt_file)
    video_parts = []
    audio_parts = []

    prev_end_ms = 0
    tasks = []

    for i, sub in enumerate(subs):
        start_ms = int(sub.start.ordinal)
        if start_ms > prev_end_ms:
            # 间隙补静音音频
            silent_audio_file = os.path.join(parts_dir, f"{base_name}_silent_{i:04d}.mp3")
            silent_duration = start_ms - prev_end_ms
            AudioSegment.silent(duration=silent_duration).export(silent_audio_file, format="mp3")
            audio_parts.append(silent_audio_file)
        tasks.append(generate_segment(video_file, sub, voice, base_name, i, parts_dir))
        prev_end_ms = int(sub.end.ordinal)

    results = await asyncio.gather(*tasks)

    for vp, ap in results:
        video_parts.append(vp)
        audio_parts.append(ap)

    # 拼接视频
    concat_video_list = os.path.join(parts_dir, "concat_video.txt")
    with open(concat_video_list, "w", encoding="utf-8") as f:
        for vp in video_parts:
            f.write(f"file '{vp}'\n")
    final_video_file = os.path.join(output_dir, f"{base_name}_tmp.mp4")
    subprocess.run([
        "ffmpeg", "-y", "-f", "concat", "-safe", "0",
        "-i", concat_video_list, "-c", "copy", final_video_file
    ], check=True)

    # 拼接音频
    concat_audio_list = os.path.join(parts_dir, "concat_audio.txt")
    with open(concat_audio_list, "w", encoding="utf-8") as f:
        for ap in audio_parts:
            f.write(f"file '{ap}'\n")
    final_audio_file = os.path.join(output_dir, f"{base_name}_tts.mp3")
    subprocess.run([
        "ffmpeg", "-y", "-f", "concat", "-safe", "0",
        "-i", concat_audio_list, "-c", "copy", final_audio_file
    ], check=True)

    # 合并最终视频+音频
    output_video_file = os.path.join(output_dir, f"{base_name}_CN.mp4")
    subprocess.run([
        "ffmpeg", "-y", "-i", final_video_file, "-i", final_audio_file,
        "-c:v", "copy", "-c:a", "aac", "-map", "0:v:0", "-map", "1:a:0",
        output_video_file
    ], check=True)
    os.remove(final_video_file)
    return output_video_file, final_audio_file

# ---------------- 9️⃣ 批量处理 ----------------
async def process_videos():
    video_files = [f for f in os.listdir(save_dir) if f.endswith(".mp4")]
    for vf in sorted(video_files):
        base = os.path.splitext(vf)[0]
        srt_file = os.path.join(save_dir, base + ".ai-zh.srt")
        if not os.path.exists(srt_file):
            print(f"❌ 跳过 {vf}，没有 SRT 文件。")
            continue

        output_video_file = os.path.join(output_dir, base + "_CN.mp4")
        output_srt = os.path.join(output_dir, base + "_CN.srt")
        if os.path.exists(output_video_file):
            print(f"⏭️ 跳过 {vf}，最终文件已存在。")
            continue

        video_file = os.path.join(save_dir, vf)
        voice = random.choice(voices)
        print(f"▶️ 正在处理 {vf}，声音: {voice}")

        final_video, final_audio = await generate_full_video(base, video_file, srt_file, voice)
        shutil.copy(srt_file, output_srt)
        print(f"✅ 完成 {vf}，输出文件位于 {output_dir}")

# ---------------- 1️⃣1️⃣ 执行 ----------------
await process_videos()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive
▶️ 正在处理 001_Udemy - Snowflake – The Complete Masterclass part1 p01 001 Welcome!.mp4，声音: zh-CN-XiaohanNeural


ConnectionTimeoutError: Connection timeout to host wss://api.msedgeservices.com/tts/cognitiveservices/websocket/v1?Ocp-Apim-Subscription-Key=6A5AA1D4EAFF4E9FB37E23D68491D6F4&ConnectionId=c752cf3382d041059feed62e3cc97af7&Sec-MS-GEC=C17B45CAD785D70E7AD430EF69F82F72D0E9A1F4FAD324739BC467E759977F52&Sec-MS-GEC-Version=1-140.0.3485.14

In [ ]:

# ---------------- 3️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 1️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import io
import asyncio
import pysrt
import edge_tts
from pydub import AudioSegment
import random
import subprocess
import shutil

# ---------------- 2️⃣ 保存目录 ----------------
save_dir = "/content/drive/MyDrive/bilibili"         # 原始目录
output_dir = os.path.join(save_dir, "output")        # 成品目录
os.makedirs(save_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# ---------------- 5️⃣ 可选声音 ----------------
voices = [
    "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "zh-CN-YunjianNeural",
    "zh-CN-YunyangNeural", "zh-CN-XiaochenNeural", "zh-CN-XiaohanNeural",
    "zh-CN-XiaomoNeural", "zh-CN-XiaoruiNeural", "zh-CN-XiaoqiuNeural",
    "zh-CN-XiaoshuangNeural", "zh-CN-XiaoyouNeural",
    "zh-CN-XiaozhenNeural", "zh-CN-YunfengNeural", "zh-CN-YunhaoNeural",
    "zh-CN-YunxiaNeural", "zh-CN-YunzeNeural", "zh-CN-XiaorouNeural",
    "zh-CN-XiaoxuanNeural", "zh-CN-XiaoyiNeural", "zh-CN-XiaomengNeural"
]

# ---------------- 6️⃣ 限流+重试生成单句 TTS ----------------
semaphore = asyncio.Semaphore(2)  # 同时最多 2 个请求

async def generate_tts_segment_with_retry(text, voice, output_path, retries=3):
    """生成单句 TTS，保存为 mp3 文件"""
    if os.path.exists(output_path):
        return  # 断点续传：已存在则跳过

    if not text.strip():
        AudioSegment.silent(duration=500).export(output_path, format="mp3")
        return

    for attempt in range(retries):
        try:
            async with semaphore:
                communicate = edge_tts.Communicate(text, voice=voice)
                audio_bytes = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_bytes += chunk["data"]
                AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3").export(output_path, format="mp3")
                return
        except Exception as e:
            if "429" in str(e):
                wait = 2 ** attempt
                print(f"⚠️ 429 限流，等待 {wait}s 重试...")
                await asyncio.sleep(wait)
            else:
                raise e
    raise Exception("TTS 多次重试失败")

# ---------------- 辅助函数：时间格式 ----------------
def srt_time_to_ms(time):
    return int(time.hours*3600*1000 + time.minutes*60*1000 + time.seconds*1000 + time.milliseconds)

# ---------------- 7️⃣ 生成音频片段并处理静音/停帧 ----------------
async def generate_full_tts_and_adjust_video(srt_path, video_file, voice, base_name):
    subs = pysrt.open(srt_path)
    clips_dir = os.path.join(output_dir, "parts", base_name)
    os.makedirs(clips_dir, exist_ok=True)

    video_segments = []
    audio_segments = []

    prev_end = 0
    for idx, sub in enumerate(subs):
        start_ms = srt_time_to_ms(sub.start)
        end_ms   = srt_time_to_ms(sub.end)
        duration_ms = end_ms - start_ms

        # 间隙处理：音频加静音
        if start_ms > prev_end:
            gap_ms = start_ms - prev_end
            audio_segments.append(AudioSegment.silent(duration=gap_ms))

        # 生成 TTS 音频
        part_file = os.path.join(clips_dir, f"{base_name}_part_{idx:04d}.mp3")
        await generate_tts_segment_with_retry(sub.text.strip(), voice, part_file)
        tts_audio = AudioSegment.from_file(part_file, format="mp3")

        # 补静音或停帧
        if len(tts_audio) < duration_ms:
            tts_audio += AudioSegment.silent(duration=(duration_ms - len(tts_audio)))
        elif len(tts_audio) > duration_ms:
            # 视频后加停帧
            extra_ms = len(tts_audio) - duration_ms
            video_segments.append((video_file, start_ms, end_ms, extra_ms))  # 带额外停帧
        else:
            video_segments.append((video_file, start_ms, end_ms, 0))

        audio_segments.append(tts_audio)
        prev_end = end_ms

    # 拼接音频
    final_audio_file = os.path.join(output_dir, f"{base_name}_tts.mp3")
    combined_audio = sum(audio_segments)
    combined_audio.export(final_audio_file, format="mp3")

    return final_audio_file, video_segments

# ---------------- 8️⃣ 视频加停帧处理 ----------------
def add_pause_frames(video_file, video_segments, output_file):
    """
    video_segments: list of tuples (video_file, start_ms, end_ms, extra_ms)
    对每段需要停帧的片段生成 ffmpeg 命令，最后 concat 合并
    """
    temp_dir = os.path.join(output_dir, "temp_video", os.path.splitext(os.path.basename(video_file))[0])
    os.makedirs(temp_dir, exist_ok=True)
    segment_files = []

    for i, (vf, start_ms, end_ms, extra_ms) in enumerate(video_segments):
        seg_file = os.path.join(temp_dir, f"seg_{i:04d}.mp4")
        # 提取原视频片段
        subprocess.run([
            "ffmpeg","-y","-i",vf,
            "-ss", str(start_ms/1000), "-to", str(end_ms/1000),
            "-c","copy", seg_file
        ])
        # 如果有 extra_ms，生成停帧
        if extra_ms > 0:
            freeze_file = os.path.join(temp_dir, f"freeze_{i:04d}.mp4")
            # 使用最后一帧生成视频
            subprocess.run([
                "ffmpeg","-y","-sseof","-0.001","-i",seg_file,
                "-t", str(extra_ms/1000),
                "-c:v","libx264","-pix_fmt","yuv420p","-an", freeze_file
            ])
            # 合并原片段 + 停帧
            combined_file = os.path.join(temp_dir, f"combined_{i:04d}.mp4")
            concat_txt = os.path.join(temp_dir, f"concat_{i:04d}.txt")
            with open(concat_txt, "w") as f:
                f.write(f"file '{seg_file}'\n")
                f.write(f"file '{freeze_file}'\n")
            subprocess.run([
                "ffmpeg","-y","-f","concat","-safe","0","-i",concat_txt,
                "-c","copy", combined_file
            ])
            segment_files.append(combined_file)
        else:
            segment_files.append(seg_file)

    # 合并所有视频片段
    concat_all = os.path.join(temp_dir, "concat_all.txt")
    with open(concat_all, "w") as f:
        for sf in segment_files:
            f.write(f"file '{sf}'\n")
    subprocess.run([
        "ffmpeg","-y","-f","concat","-safe","0","-i",concat_all,
        "-c","copy", output_file
    ])
    shutil.rmtree(temp_dir, ignore_errors=True)

# ---------------- 9️⃣ 合成视频 ----------------
def merge_video_audio_ffmpeg(video_file, audio_file, output_file):
    subprocess.run([
        "ffmpeg","-y","-i",video_file,"-i",audio_file,
        "-map","0:v:0","-map","1:a:0",
        "-c:v","copy","-c:a","aac",output_file
    ])

# ---------------- 🔟 批量处理 ----------------
async def process_videos():
    video_files = [f for f in os.listdir(save_dir) if f.endswith(".mp4")]
    for vf in sorted(video_files):
        base = os.path.splitext(vf)[0]
        srt_file = os.path.join(save_dir, base + ".ai-zh.srt")

        output_video = os.path.join(output_dir, base + "_CN.mp4")
        output_audio = os.path.join(output_dir, base + "_tts.mp3")
        output_srt   = os.path.join(output_dir, base + "_CN.srt")

        if os.path.exists(output_video):
            print(f"⏭️ 跳过 {vf}，最终文件已存在。")
            continue
        if not os.path.exists(srt_file):
            print(f"❌ 跳过 {vf}，没有 SRT 文件。")
            continue

        video_file = os.path.join(save_dir, vf)
        voice = random.choice(voices)
        print(f"▶️ 正在处理 {vf}，声音: {voice}")

        # 生成音频 + 获取视频段信息
        final_audio, video_segments = await generate_full_tts_and_adjust_video(srt_file, video_file, voice, base)

        # 生成视频停帧片段
        temp_video_file = os.path.join(output_dir, base + "_video_temp.mp4")
        add_pause_frames(video_file, video_segments, temp_video_file)

        # 合并最终视频+音频
        merge_video_audio_ffmpeg(temp_video_file, final_audio, output_video)
        os.remove(temp_video_file)

        # 复制字幕文件到 output 文件夹，并改名
        shutil.copy(srt_file, output_srt)

        print(f"✅ 完成 {vf}，输出文件位于 {output_dir}")

# ---------------- 1️⃣1️⃣ 执行 ----------------
await process_videos()

In [ ]:




import os

save_dir = "/content/drive/MyDrive/bilibili"
files = os.listdir(save_dir)

for f in files:
    if f.endswith(".srt"):
        base = f.replace(".zh-CN.ai-zh", "").replace(".ai-zh", "")
        video_name = base + "_CN.mp4"
        if video_name in files:
            new_srt = base + ".srt"
            old_path = os.path.join(save_dir, f)
            new_path = os.path.join(save_dir, new_srt)
            os.rename(old_path, new_path)
            print(f"重命名: {f} -> {new_srt}")

In [ ]:
import os
import asyncio
import pysrt
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
import subprocess
import edge_tts

# ========== 1. 生成语音 ========== #
async def generate_tts_segment_with_retry(text, voice, output_file, retries=3):
    for attempt in range(retries):
        try:
            communicate = edge_tts.Communicate(text, voice)
            await communicate.save(output_file)
            return
        except Exception as e:
            if attempt < retries - 1:
                await asyncio.sleep(2)
            else:
                raise e

# ========== 2. 截取视频并加音频 ========== #
async def process_subtitle_segment(video_file, start_ms, end_ms, audio_file, output_file):
    start = start_ms / 1000
    end = end_ms / 1000
    cmd = [
        "ffmpeg", "-y",
        "-i", video_file,
        "-ss", str(start), "-to", str(end),
        "-i", audio_file,
        "-map", "0:v", "-map", "1:a",
        "-c:v", "libx264", "-c:a", "aac",
        "-shortest", output_file
    ]
    subprocess.run(cmd, check=True)

# ========== 3. 主处理函数 ========== #
async def process_videos():
    video_file = "/content/drive/MyDrive/bilibili/input/demo.mp4"
    srt_file = "/content/drive/MyDrive/bilibili/input/demo.srt"
    output_dir = "/content/drive/MyDrive/bilibili/output"
    base = "demo"
    output_file = os.path.join(output_dir, "final_output.mp4")

    temp_audio_dir = os.path.join(output_dir, "temp_audio", base)
    temp_video_dir = os.path.join(output_dir, "temp_video", base)
    os.makedirs(temp_audio_dir, exist_ok=True)
    os.makedirs(temp_video_dir, exist_ok=True)

    subs = pysrt.open(srt_file, encoding="utf-8")
    segments = []
    voice = "zh-CN-XiaorouNeural"

    prev_end = 0  # 上一段的结束时间（ms）
    video_duration_ms = int(VideoFileClip(video_file).duration * 1000)

    # -------- 前导空白 -------- #
    if len(subs) > 0 and subs[0].start.ordinal > 0:
        gap_start = 0
        gap_end = subs[0].start.ordinal
        gap_audio = os.path.join(temp_audio_dir, f"gap_start.mp3")
        AudioSegment.silent(duration=(gap_end - gap_start)).export(gap_audio, format="mp3")
        gap_video = os.path.join(temp_video_dir, f"gap_start.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", video_file,
            "-ss", str(gap_start/1000), "-to", str(gap_end/1000),
            "-an", "-c:v", "copy", gap_video
        ])
        gap_seg = os.path.join(temp_video_dir, f"gap_start_seg.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", gap_video, "-i", gap_audio,
            "-c:v","copy","-c:a","aac", gap_seg
        ])
        segments.append(gap_seg)
        prev_end = gap_end

    # -------- 遍历字幕 -------- #
    for i, sub in enumerate(subs):
        start_ms = sub.start.ordinal
        end_ms = sub.end.ordinal

        # 防止重叠
        if start_ms < prev_end:
            start_ms = prev_end
            if start_ms >= end_ms:
                continue

        # 补 gap
        if start_ms > prev_end:
            gap_start = prev_end
            gap_end = start_ms
            gap_audio = os.path.join(temp_audio_dir, f"gap_{i:04d}.mp3")
            AudioSegment.silent(duration=(gap_end - gap_start)).export(gap_audio, format="mp3")
            gap_video = os.path.join(temp_video_dir, f"gap_{i:04d}.mp4")
            subprocess.run([
                "ffmpeg","-y","-i", video_file,
                "-ss", str(gap_start/1000), "-to", str(gap_end/1000),
                "-an", "-c:v", "copy", gap_video
            ])
            gap_seg = os.path.join(temp_video_dir, f"gap_seg_{i:04d}.mp4")
            subprocess.run([
                "ffmpeg","-y","-i", gap_video, "-i", gap_audio,
                "-c:v","copy","-c:a","aac", gap_seg
            ])
            segments.append(gap_seg)

        # 当前字幕段
        text = sub.text.strip()
        if text:
            part_file = os.path.join(temp_audio_dir, f"{i:04d}.mp3")
            await generate_tts_segment_with_retry(text, voice, part_file)

            seg_output = os.path.join(temp_video_dir, f"seg_{i:04d}.mp4")
            await process_subtitle_segment(video_file, start_ms, end_ms, part_file, seg_output)
            segments.append(seg_output)

        prev_end = end_ms

    # -------- 尾巴 -------- #
    if prev_end < video_duration_ms:
        tail_start = prev_end
        tail_end = video_duration_ms
        tail_audio = os.path.join(temp_audio_dir, "tail_silence.mp3")
        AudioSegment.silent(duration=(tail_end - tail_start)).export(tail_audio, format="mp3")
        tail_video = os.path.join(temp_video_dir, "tail_video.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", video_file,
            "-ss", str(tail_start/1000), "-to", str(tail_end/1000),
            "-an", "-c:v", "copy", tail_video
        ])
        tail_seg = os.path.join(temp_video_dir, "tail_seg.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", tail_video, "-i", tail_audio,
            "-c:v","copy","-c:a","aac", tail_seg
        ])
        segments.append(tail_seg)

    # -------- 拼接 -------- #
    concat_file = os.path.join(output_dir, f"concat_{base}.txt")
    with open(concat_file, "w") as f:
        for seg in segments:
            f.write(f"file '{seg}'\n")

    cmd_concat = [
        "ffmpeg", "-y", "-f", "concat", "-safe", "0",
        "-i", concat_file,
        "-c", "copy", output_file
    ]
    subprocess.run(cmd_concat, check=True)

    print(f"✅ 最终文件已生成: {output_file}")

# ========== 运行 ========== #
await process_videos()

In [ ]:
import os
import asyncio
import pysrt
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
import subprocess
import edge_tts

# ========== 1. 生成语音 ========== #
async def generate_tts_segment_with_retry(text, voice, output_file, retries=3):
    for attempt in range(retries):
        try:
            communicate = edge_tts.Communicate(text, voice)
            await communicate.save(output_file)
            return
        except Exception as e:
            if attempt < retries - 1:
                await asyncio.sleep(2)
            else:
                raise e

# ========== 2. 截取视频并加音频 ========== #
async def process_subtitle_segment(video_file, start_ms, end_ms, audio_file, output_file):
    start = start_ms / 1000
    end = end_ms / 1000
    cmd = [
        "ffmpeg", "-y",
        "-i", video_file,
        "-ss", str(start), "-to", str(end),
        "-i", audio_file,
        "-map", "0:v", "-map", "1:a",
        "-c:v", "libx264", "-c:a", "aac",
        "-shortest", output_file
    ]
    subprocess.run(cmd, check=True)

# ========== 3. 主处理函数 ========== #
async def process_videos():
    video_file = "/content/drive/MyDrive/bilibili/input/demo.mp4"
    srt_file = "/content/drive/MyDrive/bilibili/input/demo.srt"
    output_file = "/content/drive/MyDrive/bilibili/output/final_output.mp4"
    temp_audio_dir = "/content/drive/MyDrive/bilibili/output/temp_audio"
    os.makedirs(temp_audio_dir, exist_ok=True)

    subs = pysrt.open(srt_file, encoding="utf-8")
    segments = []
    voice = "zh-CN-XiaorouNeural"

    # -------- 每个字幕片段 -------- #
    for i, sub in enumerate(subs):
        text = sub.text.strip()
        if not text:
            continue

        part_file = os.path.join(temp_audio_dir, f"{i:04d}.mp3")
        await generate_tts_segment_with_retry(text, voice, part_file)

        seg_output = os.path.join(temp_audio_dir, f"seg_{i:04d}.mp4")
        start_ms = sub.start.ordinal
        end_ms = sub.end.ordinal
        #await process_subtitle_segment(video_file, start_ms, end_ms, part_file, seg_output)
        #segments.append(seg_output)

        # -------- 补字幕间隙 -------- #
        if i > 0 and start_ms > subs[i-1].end.ordinal:
            gap_audio_file = os.path.join(temp_audio_dir, f"gap_{i:04d}.mp3")
            gap_duration = start_ms - subs[i-1].end.ordinal
            AudioSegment.silent(duration=gap_duration).export(gap_audio_file, format="mp3")
            gap_output = os.path.join(temp_audio_dir, f"gap_{i:04d}.mp4")
            await process_subtitle_segment(video_file, subs[i-1].end.ordinal, start_ms, gap_audio_file, gap_output)
            segments.append(gap_output)

        await process_subtitle_segment(video_file, start_ms, end_ms, part_file, seg_output)
        segments.append(seg_output)

    # -------- 补最后的尾巴 -------- #
    video_duration = VideoFileClip(video_file).duration * 1000
    last_end_ms = subs[-1].end.ordinal if subs else 0

    if last_end_ms < video_duration:
        tail_audio_file = os.path.join(temp_audio_dir, "tail_silence.mp3")
        tail_duration = video_duration - last_end_ms
        AudioSegment.silent(duration=tail_duration).export(tail_audio_file, format="mp3")
        tail_output = os.path.join(temp_audio_dir, "tail_segment.mp4")
        await process_subtitle_segment(video_file, last_end_ms, video_duration, tail_audio_file, tail_output)
        segments.append(tail_output)

    # -------- 拼接所有片段 -------- #
    concat_file = os.path.join(temp_audio_dir, "concat.txt")
    with open(concat_file, "w") as f:
        for seg in segments:
            f.write(f"file '{seg}'\n")

    cmd_concat = [
        "ffmpeg", "-y", "-f", "concat", "-safe", "0",
        "-i", concat_file,
        "-c", "copy", output_file
    ]
    subprocess.run(cmd_concat, check=True)

    print(f"✅ 最终文件已生成: {output_file}")

# ========== 运行 ========== #
await process_videos()

In [ ]:

import os
import asyncio
import pysrt
from pydub import AudioSegment
from moviepy.editor import VideoFileClip
import subprocess
import edge_tts
import random  # ✅ 新增

# ========== 1. 生成语音 ========== #
async def generate_tts_segment_with_retry(text, voice, output_file, retries=3):
    for attempt in range(retries):
        try:
            communicate = edge_tts.Communicate(text, voice)
            await communicate.save(output_file)
            return
        except Exception as e:
            if attempt < retries - 1:
                await asyncio.sleep(2)
            else:
                raise e

# ========== 2. 截取视频并加音频 ========== #
async def process_subtitle_segment(video_file, start_ms, end_ms, audio_file, output_file):
    start = start_ms / 1000
    end = end_ms / 1000
    cmd = [
        "ffmpeg", "-y",
        "-i", video_file,
        "-ss", str(start), "-to", str(end),
        "-i", audio_file,
        "-map", "0:v", "-map", "1:a",
        "-c:v", "libx264", "-c:a", "aac",
        "-shortest", output_file
    ]
    subprocess.run(cmd, check=True)

# ========== 3. 主处理函数 ========== #
async def process_videos():
    video_file = "/content/drive/MyDrive/bilibili/input/demo.mp4"
    srt_file = "/content/drive/MyDrive/bilibili/input/demo.srt"
    output_dir = "/content/drive/MyDrive/bilibili/output"
    base = "demo"
    output_file = os.path.join(output_dir, "final_output.mp4")

    temp_audio_dir = os.path.join(output_dir, "temp_audio", base)
    temp_video_dir = os.path.join(output_dir, "temp_video", base)
    os.makedirs(temp_audio_dir, exist_ok=True)
    os.makedirs(temp_video_dir, exist_ok=True)

    subs = pysrt.open(srt_file, encoding="utf-8")
    segments = []

    # ✅ 声音列表（可以自己扩充）
    voices = [
        "zh-CN-XiaorouNeural",
        "zh-CN-XiaoqiuNeural",
        "zh-CN-YunxiNeural",
        "zh-CN-YunyangNeural",
        "zh-CN-XiaohanNeural"
    ]

    prev_end = 0  # 上一段的结束时间（ms）
    video_duration_ms = int(VideoFileClip(video_file).duration * 1000)

    # -------- 前导空白 -------- #
    if len(subs) > 0 and subs[0].start.ordinal > 0:
        gap_start = 0
        gap_end = subs[0].start.ordinal
        gap_audio = os.path.join(temp_audio_dir, f"gap_start.mp3")
        AudioSegment.silent(duration=(gap_end - gap_start)).export(gap_audio, format="mp3")
        gap_video = os.path.join(temp_video_dir, f"gap_start.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", video_file,
            "-ss", str(gap_start/1000), "-to", str(gap_end/1000),
            "-an", "-c:v", "copy", gap_video
        ])
        gap_seg = os.path.join(temp_video_dir, f"gap_start_seg.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", gap_video, "-i", gap_audio,
            "-c:v","copy","-c:a","aac", gap_seg
        ])
        segments.append(gap_seg)
        prev_end = gap_end

    # -------- 遍历字幕 -------- #
    for i, sub in enumerate(subs):
        start_ms = sub.start.ordinal
        end_ms = sub.end.ordinal

        # 防止重叠
        if start_ms < prev_end:
            start_ms = prev_end
            if start_ms >= end_ms:
                continue

        # 补 gap
        if start_ms > prev_end:
            gap_start = prev_end
            gap_end = start_ms
            gap_audio = os.path.join(temp_audio_dir, f"gap_{i:04d}.mp3")
            AudioSegment.silent(duration=(gap_end - gap_start)).export(gap_audio, format="mp3")
            gap_video = os.path.join(temp_video_dir, f"gap_{i:04d}.mp4")
            subprocess.run([
                "ffmpeg","-y","-i", video_file,
                "-ss", str(gap_start/1000), "-to", str(gap_end/1000),
                "-an", "-c:v", "copy", gap_video
            ])
            gap_seg = os.path.join(temp_video_dir, f"gap_seg_{i:04d}.mp4")
            subprocess.run([
                "ffmpeg","-y","-i", gap_video, "-i", gap_audio,
                "-c:v","copy","-c:a","aac", gap_seg
            ])
            segments.append(gap_seg)

        # 当前字幕段
        text = sub.text.strip()
        if text:
            # ✅ 随机选择一个声音
            voice = random.choice(voices)
            print(f"🎙 使用声音: {voice} -> {text}")

            part_file = os.path.join(temp_audio_dir, f"{i:04d}.mp3")
            await generate_tts_segment_with_retry(text, voice, part_file)

            seg_output = os.path.join(temp_video_dir, f"seg_{i:04d}.mp4")
            await process_subtitle_segment(video_file, start_ms, end_ms, part_file, seg_output)
            segments.append(seg_output)

        prev_end = end_ms

    # -------- 尾巴 -------- #
    if prev_end < video_duration_ms:
        tail_start = prev_end
        tail_end = video_duration_ms
        tail_audio = os.path.join(temp_audio_dir, "tail_silence.mp3")
        AudioSegment.silent(duration=(tail_end - tail_start)).export(tail_audio, format="mp3")
        tail_video = os.path.join(temp_video_dir, "tail_video.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", video_file,
            "-ss", str(tail_start/1000), "-to", str(tail_end/1000),
            "-an", "-c:v", "copy", tail_video
        ])
        tail_seg = os.path.join(temp_video_dir, "tail_seg.mp4")
        subprocess.run([
            "ffmpeg","-y","-i", tail_video, "-i", tail_audio,
            "-c:v","copy","-c:a","aac", tail_seg
        ])
        segments.append(tail_seg)

    # -------- 拼接 -------- #
    concat_file = os.path.join(output_dir, f"concat_{base}.txt")
    with open(concat_file, "w") as f:
        for seg in segments:
            f.write(f"file '{seg}'\n")

    cmd_concat = [
        "ffmpeg", "-y", "-f", "concat", "-safe", "0",
        "-i", concat_file,
        "-c", "copy", output_file
    ]
    subprocess.run(cmd_concat, check=True)

    print(f"✅ 最终文件已生成: {output_file}")

# ========== 运行 ========== #
await process_videos()

In [ ]:
# ---------------- 3️⃣ 安装依赖 ----------------
!pip install yt-dlp edge-tts pysrt pydub
!apt-get install ffmpeg -y

# ---------------- 1️⃣ 挂载 Google Drive ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import io
import asyncio
import pysrt
import edge_tts
from pydub import AudioSegment
import random
import subprocess
import shutil

# ---------------- 2️⃣ 保存目录 ----------------
save_dir = "/content/drive/MyDrive/bilibili"         # 原始目录
output_dir = os.path.join(save_dir, "output")        # 成品目录
os.makedirs(save_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# ---------------- 5️⃣ 可选声音 ----------------
voices = [
    "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "zh-CN-YunjianNeural",
    "zh-CN-YunyangNeural", "zh-CN-XiaochenNeural", "zh-CN-XiaohanNeural",
    "zh-CN-XiaomoNeural", "zh-CN-XiaoruiNeural", "zh-CN-XiaoqiuNeural",
    "zh-CN-XiaoshuangNeural", "zh-CN-XiaoyouNeural",
    "zh-CN-XiaozhenNeural", "zh-CN-YunfengNeural", "zh-CN-YunhaoNeural",
    "zh-CN-YunxiaNeural", "zh-CN-YunzeNeural", "zh-CN-XiaorouNeural",
    "zh-CN-XiaoxuanNeural", "zh-CN-XiaoyiNeural", "zh-CN-XiaomengNeural"
]

# ---------------- 6️⃣ 限流+重试生成单句 TTS ----------------
semaphore = asyncio.Semaphore(2)  # 同时最多 2 个请求

async def generate_tts_segment_with_retry(text, voice, output_path, retries=3):
    """生成单句 TTS，保存为 mp3 文件"""
    if os.path.exists(output_path):
        return  # 断点续传：已存在则跳过

    if not text.strip():
        AudioSegment.silent(duration=500).export(output_path, format="mp3")
        return

    for attempt in range(retries):
        try:
            async with semaphore:
                communicate = edge_tts.Communicate(text, voice=voice)
                audio_bytes = b""
                async for chunk in communicate.stream():
                    if chunk["type"] == "audio":
                        audio_bytes += chunk["data"]
                AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3").export(output_path, format="mp3")
                return
        except Exception as e:
            if "429" in str(e):
                wait = 2 ** attempt
                print(f"⚠️ 429 限流，等待 {wait}s 重试...")
                await asyncio.sleep(wait)
            else:
                raise e
    raise Exception("TTS 多次重试失败")

# ---------------- 辅助函数：时间格式 ----------------
def srt_time_to_ms(time):
    return int(time.hours*3600*1000 + time.minutes*60*1000 + time.seconds*1000 + time.milliseconds)

# ---------------- 7️⃣ 生成音频片段并处理静音/停帧 ----------------
async def generate_full_tts_and_adjust_video(srt_path, video_file, base_name):
    subs = pysrt.open(srt_path)
    clips_dir = os.path.join(output_dir, "parts", base_name)
    os.makedirs(clips_dir, exist_ok=True)

    video_segments = []
    audio_segments = []

    # 获取视频总长度（ms）
    result = subprocess.run([
        "ffprobe", "-v", "error", "-select_streams", "v:0",
        "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", video_file
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    video_duration_ms = int(float(result.stdout) * 1000)

    prev_start = 0
    prev_end = 0
    speed_factor = 0
    gap_ms = 0
    # 随机选择声音生成 TTS
    voice = random.choice(voices)

    for idx, sub in enumerate(subs):
        start_ms = srt_time_to_ms(sub.start)
        end_ms   = srt_time_to_ms(sub.end)
        duration_ms = end_ms - start_ms

        # 前端或中间空白，音频加静音
        if start_ms > prev_end:
            gap_ms = start_ms - prev_end
            audio_segments.append(AudioSegment.silent(duration=gap_ms))

            start_ms = prev_end
            duration_ms = end_ms - prev_end

        # 随机选择声音生成 TTS
        #voice = random.choice(voices)
        part_file = os.path.join(clips_dir, f"{base_name}_part_{idx:04d}.mp3")
        await generate_tts_segment_with_retry(sub.text.strip(), voice, part_file)
        tts_audio = AudioSegment.from_file(part_file, format="mp3")

        # 音频比字幕段短 → 补静音；比字幕段长 → 视频末尾加停帧
        if len(tts_audio) < duration_ms:
            tts_audio += AudioSegment.silent(duration=(duration_ms - len(tts_audio)))
            video_segments.append((video_file, start_ms, end_ms, 0))
        elif len(tts_audio) > duration_ms:
            #extra_ms = len(tts_audio) - duration_ms
            speed_factor = duration_ms / len(tts_audio)

            video_segments.append((video_file, start_ms, end_ms, speed_factor))
        else:
            video_segments.append((video_file, start_ms, end_ms, 0))

        audio_segments.append(tts_audio)
        prev_end = end_ms
        prev_start = start_ms

    # 结尾空白处理
    if prev_end < video_duration_ms:
        gap_ms = video_duration_ms - prev_end
        if gap_ms < 300:
            extra_ms = 300 - gap_ms
            gap_ms = 300
        else:
            extra_ms = 0
        audio_segments.append(AudioSegment.silent(duration=gap_ms))
        # 视频保持原始，不需停帧
        video_segments.append((video_file, prev_end, video_duration_ms, extra_ms))

    # 拼接音频
    final_audio_file = os.path.join(output_dir, f"{base_name}_tts.mp3")
    combined_audio = sum(audio_segments)
    combined_audio.export(final_audio_file, format="mp3")

    return final_audio_file, video_segments

# ---------------- 8️⃣ 视频加停帧处理 ---------------- #
def add_pause_frames(video_file, video_segments, output_file):
    temp_dir = os.path.join(output_dir, "temp_video", os.path.splitext(os.path.basename(video_file))[0])
    os.makedirs(temp_dir, exist_ok=True)
    segment_files = []

    for i, (vf, start_ms, end_ms, speed_factor) in enumerate(video_segments):
        seg_file = os.path.join(temp_dir, f"seg_{i:04d}.mp4")

        duration_ms = end_ms - start_ms

        #print(f"Start:{str(start_ms)};End:{str(end_ms)};duration_ms:{str(duration_ms)}; extra_ms:{str(extra_ms)};FILE:{seg_file}")
        print(f"Start:{str(start_ms)};End:{str(end_ms)};duration_ms:{str(duration_ms)}; speed_factor:{str(speed_factor)};FILE:{seg_file}")
        # 截取视频段并重新编码，保证短片段不会黑屏
        subprocess.run([
            "ffmpeg", "-y",
            "-i", vf,
            "-ss", str(start_ms/1000),
            "-to", str(end_ms/1000),
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            "-an",
            seg_file
        ])

        if speed_factor > 0:
            freeze_file = os.path.join(temp_dir, f"freeze_{i:04d}.mp4")
            # 添加停帧
            """
            subprocess.run([
                "ffmpeg", "-y",
                "-i", seg_file,
                "-vf", f"tpad=stop_mode=clone:stop_duration={extra_ms/1000}",
                "-c:v", "libx264",
                "-pix_fmt", "yuv420p",
                "-an",  # 不保留原音频
                freeze_file
            ])
            """
            subprocess.run([
                "ffmpeg", "-y",
                "-i", seg_file,
                "-vf", f"setpts={1/speed_factor}*PTS",
                "-c:v", "libx264",
                "-pix_fmt", "yuv420p",
                "-an",
                freeze_file
            ])

            segment_files.append(freeze_file)
        else:
            segment_files.append(seg_file)

    # 拼接所有段落
    concat_all = os.path.join(temp_dir, "concat_all.txt")
    with open(concat_all, "w") as f:
        for sf in segment_files:
            f.write(f"file '{sf}'\n")

    subprocess.run([
        "ffmpeg", "-y", "-f", "concat", "-safe", "0", "-i", concat_all,
        "-c:v", "libx264", "-pix_fmt", "yuv420p", "-c:a", "aac",
        output_file
    ])


def merge_video_audio_ffmpeg(video_file, audio_file, output_file):
    subprocess.run([
        "ffmpeg","-y","-i",video_file,"-i",audio_file,
        "-map","0:v:0","-map","1:a:0",
        "-c:v","libx264","-pix_fmt","yuv420p","-c:a","aac", output_file
    ])

# ---------------- 🔟 批量处理 ----------------
async def process_videos():
    video_files = [f for f in os.listdir(save_dir) if f.endswith(".mp4")]
    for vf in sorted(video_files):
        base = os.path.splitext(vf)[0]
        srt_file = os.path.join(save_dir, base + ".ai-zh.srt")

        output_video = os.path.join(output_dir, base + "_CN.mp4")
        output_audio = os.path.join(output_dir, base + "_tts.mp3")
        output_srt   = os.path.join(output_dir, base + "_CN.srt")

        if os.path.exists(output_video):
            print(f"⏭️ 跳过 {vf}，最终文件已存在。")
            continue
        if not os.path.exists(srt_file):
            print(f"❌ 跳过 {vf}，没有 SRT 文件。")
            continue

        video_file = os.path.join(save_dir, vf)
        print(f"▶️ 正在处理 {vf}")

        # 生成音频 + 视频段信息
        final_audio, video_segments = await generate_full_tts_and_adjust_video(srt_file, video_file, base)

        # 生成视频停帧片段
        temp_video_file = os.path.join(output_dir, base + "_video_temp.mp4")
        add_pause_frames(video_file, video_segments, temp_video_file)

        # 合并最终视频 + 音频
        merge_video_audio_ffmpeg(temp_video_file, final_audio, output_video)
        #os.remove(temp_video_file)

        # 复制字幕文件
        shutil.copy(srt_file, output_srt)
        print(f"✅ 完成 {vf}，输出文件位于 {output_dir}")

# ---------------- 1️⃣1️⃣ 执行 ----------------
await process_videos()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive
▶️ 正在处理 001_Udemy - Snowflake – The Complete Masterclass part1 p01 001 Welcome!.mp4
Start:0;End:1960;duration_ms:1960; speed_factor:0.6481481481481481;FILE:/content/drive/MyDrive/bilibili/output/temp_video/001_Udemy - Snowflake – The Complete Masterclass part1 p01 001 Welcome!/seg_0000.mp4
Start:1960;End:3480;duration_ms:1520; speed_factor:0.6209150326797386;FILE:/content/drive/MyDrive/bilibili/output/temp_video/001_Udemy - Snowflake – The Complete Masterclass part1 p01 001 Welcome!/seg_0001.mp4
Start:3480;End:7400;duration_ms:3920; speed_factor:0.928030303030303;FILE:/content/drive/MyDrive/bilibili/output/temp_video/001_Udemy - Snowflake – The Complete Masterclass part1 p01 001 Welcome!/seg_0002.mp4
Start:7400;End:10240;duration_ms:2840;